## Notes

DETR-run5 - Accuracy = 94.8%

DETR-run7 - Accuracy = 84.5%

In [1]:
import os 
import supervision as sv
from transformers import DetrForObjectDetection, DetrImageProcessor
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from PIL import Image, ImageDraw, ImageFont
from inference2 import loadModel, inference4, inference2, inference3
from img2pdf import readPDF, savePDF
from createGroundTruth import create_ground_truth_dict
import time
import torchvision
from torchvision.ops import box_iou

import cv2
import random
import numpy as np
import pandas as pd

d:\Object Detection\object_detection\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

In [4]:
## CocoDetection Class 

# settings
ANNOTATION_FILE_NAME = r"result.json"
TRAIN_DIRECTORY = os.path.join(r"dataset2", r"train")
VAL_DIRECTORY = os.path.join(r"dataset2", r"val")
TEST_DIRECTORY = os.path.join(r"dataset2", r"test")

class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(
        self,
        image_directory_path: str,
        image_processor,
        train: bool = True
    ):
        annotation_file_path = os.path.join(image_directory_path, ANNOTATION_FILE_NAME)
        super(CocoDetection, self).__init__(image_directory_path, annotation_file_path)
        self.image_processor = image_processor

    def __getitem__(self, idx):
        images, annotations = super(CocoDetection, self).__getitem__(idx)
        image_id = self.ids[idx]
        annotations = {'image_id': image_id, 'annotations': annotations}
        encoding = self.image_processor(images=images, annotations=annotations, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze()
        target = encoding["labels"][0]

        return pixel_values, target
    
TRAIN_DATASET = CocoDetection(
    image_directory_path=TRAIN_DIRECTORY,
    image_processor=image_processor,
    train=True)
VAL_DATASET = CocoDetection(
    image_directory_path=VAL_DIRECTORY,
    image_processor=image_processor,
    train=False)
TEST_DATASET = CocoDetection(
    image_directory_path=TEST_DIRECTORY,
    image_processor=image_processor,
    train=False)

print("Number of training examples:", len(TRAIN_DATASET))
print("Number of validation examples:", len(VAL_DATASET))
print("Number of test examples:", len(TEST_DATASET))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Number of training examples: 130
Number of validation examples: 32
Number of test examples: 19


In [68]:
CHECKPOINT = "facebook/detr-resnet-50"

# Best Performing Model
MODEL_PATH = "models/DETR-run7"

# Doesnt Work
# MODEL_101 = 'facebook/detr-resnet-101'
# CHECKPOINT_101 = 'facebook/detr-resnet-101'

# Older Model
# MODEL_PATH = "models/DETR-run4"


## Load Model
def loadModel(MODEL_PATH, CHECKPOINT):
    model = DetrForObjectDetection.from_pretrained(MODEL_PATH)
    image_processor = DetrImageProcessor.from_pretrained(CHECKPOINT)
    return model, image_processor

In [69]:
from transformers import DetrForObjectDetection
import torch
from collections import OrderedDict

# Initialize the model architecture
model, image_processor = loadModel(MODEL_PATH=MODEL_PATH, CHECKPOINT=CHECKPOINT)

In [77]:

# model, image_processor = loadModel(MODEL_PATH=MODEL_101, CHECKPOINT=CHECKPOINT_101)

# Load your checkpoint
checkpoint = torch.load("models/DETR-run7/ModelCheckpoints2/detr-epoch=47-val_loss=0.53.ckpt", map_location='cpu')

# Get the state dict
state_dict = checkpoint['state_dict']

# Remove the 'model.model.' prefix from the state dict keys
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k.replace("model.model.", "")
    new_state_dict[name] = v

# Load the modified state dict
model.load_state_dict(new_state_dict, strict=False)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("Model loaded successfully!")

Model loaded successfully!


In [78]:
## Create Ground Truth Dictionary
# ground_truth = create_ground_truth_dict('dataset2/test/result.json')

# For Older models
ground_truth = create_ground_truth_dict('dataset2/test/result.json')

In [79]:
categories = TEST_DATASET.coco.cats
id2label = {k: v['name'] for k,v in categories.items()}
id2label

{0: 'bar-scale', 1: 'color-stamp', 2: 'detail-labels', 3: 'north-sign'}

In [80]:
def add_missing_label(image, save_path, label):
    id2label = {0: 'bar-scale', 1: 'color-stamp', 2: 'detail-labels', 3: 'north-sign'}
    # id2label = {4: 'north-sign', 2: 'color-stamp', 1: 'bar-scale', 3: 'detail-labels'}
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    if label:
        text = ""
        for i in label: 
            text = text + f"{id2label[i]} not detected" + "\n"
    else:
        text = ""
    position = (10, 10)
    draw.text(position, text, fill="red", font=font)
    image.save(save_path)

In [81]:
## Inference Function
IMAGE_FOLDER = 'Temp/images'
CONFIDENCE_THRESHOLD = 0.6
IOU_THRESHOLD = 0.7

def inference(image_folder, CONFIDENCE_THRESHOLD, IOU_THRESHOLD):
    results_dict = {}
    
    for img in os.listdir(image_folder):
        IMAGE_PATH = os.path.join(image_folder, img)
        print(f"Processing {IMAGE_PATH}")

        image = cv2.imread(IMAGE_PATH)
        inputs = image_processor(images=image, return_tensors='pt')

        # Move inputs to the same device as the model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        # Get ground truth for this image
        target = ground_truth.get(img, {'boxes': torch.empty((0, 4)), 'labels': torch.empty((0,), dtype=torch.long)})
        target = {k: v for k, v in target.items()}
        # print(target)
        with torch.no_grad():
            outputs = model(**inputs)
            
            # Post-process
            target_sizes = torch.tensor([image.shape[:2]]).to(model.device)
            results = image_processor.post_process_object_detection(
                outputs=outputs,
                threshold=CONFIDENCE_THRESHOLD,
                target_sizes=target_sizes
            )[0]
        
        
        detections = sv.Detections.from_transformers(transformers_results=results).with_nms(IOU_THRESHOLD)
        labels = [f"{id2label[class_id]} {confidence:.2f}" for _, confidence, class_id, _ in detections]
        
        print(f"Detected Labels - {set(detections.class_id)}") 
        box_annotator = sv.BoxAnnotator()
        frame = box_annotator.annotate(scene=image, detections=detections, labels=labels)
        
        image = Image.fromarray(frame)
        image_path = f"Temp/results/annotated_{img}"
        all_labels = {0, 1, 2, 3}
        label = all_labels - set(detections.class_id)
        # image.save(image_path)
        add_missing_label(image, image_path, label) # type: ignore
        results_dict[IMAGE_PATH.replace('Temp/', '')] = results
    return results_dict

In [82]:
start = time.time()
results = inference(IMAGE_FOLDER, 0.5, 0.5)
end = time.time()

print(f"Time taken: {end - start:.2f} seconds")

Processing Temp/images\0f7db672-drawing_88.png
Detected Labels - {1, 2, 3}
Processing Temp/images\1eecb4f3-drawing_25.png
Detected Labels - {0, 1, 2, 3}
Processing Temp/images\2832a63f-drawing_44.png
Detected Labels - {0, 1, 2, 3}
Processing Temp/images\2e4aefb0-drawing_15.png
Detected Labels - {0, 1, 2, 3}
Processing Temp/images\3c56fa90-drawing_7.png
Detected Labels - {1, 2, 3}
Processing Temp/images\594b3b57-drawing_113.png
Detected Labels - {0, 2, 3}
Processing Temp/images\5b5d1b9b-drawing_123.png
Detected Labels - {1, 2, 3}
Processing Temp/images\72719062-drawing_43.png
Detected Labels - {0, 1, 2, 3}
Processing Temp/images\777ab86f-drawing_22.png
Detected Labels - {1, 2, 3}
Processing Temp/images\78e13516-drawing_45.png
Detected Labels - {0, 1, 2, 3}
Processing Temp/images\817c3fb2-drawing_42.png
Detected Labels - {0, 1, 2, 3}
Processing Temp/images\81c64eea-drawing_106.png
Detected Labels - {0, 1, 2, 3}
Processing Temp/images\8e605e02-drawing_56.png
Detected Labels - {0, 1, 2, 3}

In [15]:
def savePDF(image_dir, pdf_path):
    imagelist = []
    im = None
    for path in os.listdir(image_dir):
        im = Image.open(os.path.join(image_dir, path)).convert('RGB')
        imagelist.append(im)
    # im3 = image3.convert('RGB')

    im.save(pdf_path,save_all=True, append_images=imagelist)

In [448]:
savePDF(image_dir='Temp/results', pdf_path='output.pdf')

### Create Predictions DataFrame

In [16]:
predictions_df = pd.DataFrame()
for k in results:
    df = pd.DataFrame(results[k]['boxes'].detach().to('cpu').numpy(), columns=['x1', 'y1', 'x2', 'y2'])
    df['labels'] = results[k]['labels'].detach().to('cpu').numpy()
    df['image'] = k.replace('images\\', '')
    # print(df['image'])
    predictions_df = pd.concat([predictions_df, df], ignore_index=True)
predictions_df.shape

(124, 6)

In [17]:
predictions_df['labels'].value_counts()

labels
2    49
1    32
3    22
0    21
Name: count, dtype: int64

### Create Ground Truths DataFrame

In [18]:
ground_truth_df = pd.DataFrame()
for k in ground_truth:
    df = pd.DataFrame(ground_truth[k]['boxes'], columns=['x1', 'y1', 'x2', 'y2'])
    df['labels'] = ground_truth[k]['labels']
    # print(k)
    df['image'] = k.replace('images/', '')
    # print(df['image'])
    ground_truth_df = pd.concat([ground_truth_df, df], ignore_index=True)
    
ground_truth_df.shape

(116, 6)

In [19]:
ground_truth_df['labels'].value_counts()

labels
2    50
0    27
3    22
1    17
Name: count, dtype: int64

## Sort DFs

In [20]:
ground_truth_df = ground_truth_df.sort_values(by=['image', 'labels', 'y1', 'x1'])
predictions_df = predictions_df.sort_values(by=['image', 'labels', 'y1', 'x1'])

### Row Count

In [21]:
ground_truth_df['row_count'] = ground_truth_df.groupby(['image', 'labels']).cumcount() + 1
predictions_df['row_count'] = predictions_df.groupby(['image', 'labels']).cumcount() + 1

In [22]:
# ground_truth_df.head(10)
predictions_df.head(10)

,x1,y1,x2,y2,labels,image,row_count
2,281.503082,531.329590,359.037598,558.791687,1,11913d58-drawing_88.png,1
0,279.982239,533.085571,364.689423,560.504028,1,11913d58-drawing_88.png,2
3,627.046082,263.431274,713.595642,293.772980,2,11913d58-drawing_88.png,1
4,46.165363,68.864998,86.792572,104.516121,3,11913d58-drawing_88.png,1
1,36.104023,352.726593,78.724777,387.728058,3,11913d58-drawing_88.png,2
8,43.277252,510.151123,138.285721,537.268921,0,17bd5f92-drawing_106.png,1
7,613.102661,384.311493,706.184570,423.056732,1,17bd5f92-drawing_106.png,1
11,617.153931,384.510895,705.557861,419.803040,1,17bd5f92-drawing_106.png,2
5,610.900513,386.051361,697.938904,423.295135,1,17bd5f92-drawing_106.png,3
6,425.357208,105.869209,499.823517,138.254074,2,17bd5f92-drawing_106.png,1


## Merge The DataFrames for Comparison

In [23]:
merged_df = pd.merge(ground_truth_df, predictions_df, on=['image', 'labels', 'row_count'], how='outer', suffixes=('_gt', '_pred'), validate='many_to_many')

In [24]:
merged_df.head(12)

,x1_gt,y1_gt,x2_gt,y2_gt,labels,image,row_count,x1_pred,y1_pred,x2_pred,y2_pred
0,626.232558,409.581395,732.186047,429.348837,0,11913d58-drawing_88.png,1,NaN,NaN,NaN,NaN
1,279.116279,529.767442,363.720930,559.023256,1,11913d58-drawing_88.png,1,281.503082,531.329590,359.037598,558.791687
2,NaN,NaN,NaN,NaN,1,11913d58-drawing_88.png,2,279.982239,533.085571,364.689423,560.504028
3,626.232558,266.465116,713.209302,290.186047,2,11913d58-drawing_88.png,1,627.046082,263.431274,713.595642,293.772980
4,217.441860,518.697674,278.325581,542.418605,2,11913d58-drawing_88.png,2,NaN,NaN,NaN,NaN
5,46.651163,66.418605,83.813953,105.162791,3,11913d58-drawing_88.png,1,46.165363,68.864998,86.792572,104.516121
6,39.534884,351.069767,75.116279,389.813953,3,11913d58-drawing_88.png,2,36.104023,352.726593,78.724777,387.728058
7,46.651163,515.534884,136.790698,535.302326,0,17bd5f92-drawing_106.png,1,43.277252,510.151123,138.285721,537.268921
8,617.534884,386.651163,695.813953,419.860465,1,17bd5f92-drawing_106.png,1,613.102661,384.311493,706.184570,423.056732
9,NaN,NaN,NaN,NaN,1,17bd5f92-drawing_106.png,2,617.153931,384.510895,705.557861,419.803040


## Calculate IOU

In [25]:
def calculate_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.
    
    Parameters:
    - box1: (x1, y1, x2, y2) coordinates of the first bounding box
    - box2: (x1, y1, x2, y2) coordinates of the second bounding box
    
    Returns:
    - iou: Intersection over Union (IoU) value
    """
    # Unpack the coordinates of the two boxes
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # Calculate the (x, y) coordinates of the intersection rectangle
    xi1 = max(x1_1, x1_2)
    yi1 = max(y1_1, y1_2)
    xi2 = min(x2_1, x2_2)
    yi2 = min(y2_1, y2_2)
    
    # Calculate the area of the intersection rectangle
    inter_width = max(0, xi2 - xi1)
    inter_height = max(0, yi2 - yi1)
    inter_area = inter_width * inter_height
    
    # Calculate the area of both bounding boxes
    box1_area = (x2_1 - x1_1) * (y2_1 - y1_1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # Calculate the union area
    union_area = box1_area + box2_area - inter_area
    
    # Calculate the IoU
    iou = inter_area / union_area if union_area != 0 else 0
    
    return iou

In [26]:
merged_df['iou'] = merged_df.apply(lambda x: calculate_iou((x['x1_gt'], x['y1_gt'], x['x2_gt'], x['y2_gt']), (x['x1_pred'], x['y1_pred'], x['x2_pred'], x['y2_pred'])), axis=1)

## Calculate TP, FP, FN

In [27]:
TP, FP, FN = 0, 0, 0
merged_df['result'] = ''
for i in range(merged_df.shape[0]):
    if merged_df['iou'][i] > 0.6:
        TP += 1
        merged_df.loc[i, 'result'] = 'TP'
        
    elif merged_df['iou'][i] == 0:
        FP += 1
        merged_df.loc[i, 'result'] = 'FP'
    
    elif pd.isna(merged_df['iou'][i]):
        if pd.isna(merged_df['x1_pred'][i]):
            FN += 1
            merged_df.loc[i, 'result'] = 'FN'
        if pd.isna(merged_df['x1_gt'][i]):
            FP += 1
            merged_df.loc[i, 'result'] = 'FP'
    
    else: 
        FP += 1
        merged_df.loc[i, 'result'] = 'FP'
    

In [28]:
print("True Positive - ", TP)
print("False Positive - ", FP)
print("False Negative - ", FN)

True Positive -  79
False Positive -  45
False Negative -  15


In [29]:
print("Accuracy - ", TP/(TP+FP+FN))
print("Precision - ", TP/(TP+FP))
print("Recall - ", TP/(TP+FN))

Accuracy -  0.5683453237410072
Precision -  0.6370967741935484
Recall -  0.8404255319148937


## Class Wise Accuracy

### Bar Scale (1)

In [30]:
id2label

{0: 'bar-scale', 1: 'color-stamp', 2: 'detail-labels', 3: 'north-sign'}

In [31]:
merged_df.loc[merged_df['labels']==0]['result'].value_counts()

result
TP    13
FN     8
FP     8
Name: count, dtype: int64

In [32]:
TP, FP = merged_df.loc[merged_df['labels']==0]['result'].value_counts()
FN = 0
# TP, FN, FP = merged_df.loc[merged_df['labels']==1]['result'].value_counts()

print(f"Accuracy = {TP/(TP+FP+FN):.4f}")
print(f"Precision = {TP/(TP+FP):.4f}")
print(f"Recall = {TP/(TP+FN):.4f}")

ValueError: too many values to unpack (expected 2)

### Color Stamp (2)

In [ ]:
merged_df.loc[merged_df['labels']==1]['result'].value_counts()

result
TP    17
Name: count, dtype: int64

In [ ]:
TP = merged_df.loc[merged_df['labels']==1]['result'].value_counts()
FP, FN = 0, 0

print(f"Accuracy = {(TP[0]/(TP[0]+FP+FN)):.4f}")
print(f"Precision = {TP[0]/(TP[0]+FP):.4f}")
print(f"Recall = {TP[0]/(TP[0]+FN):.4f}")

Accuracy = 1.0000
Precision = 1.0000
Recall = 1.0000


C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15412\715305068.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Accuracy = {(TP[0]/(TP[0]+FP+FN)):.4f}")
C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15412\715305068.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Precision = {TP[0]/(TP[0]+FP):.4f}")
C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15412\715305068.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a va

### Detail Labels (3)

In [ ]:
TP, FP, FN = merged_df.loc[merged_df['labels']==2]['result'].value_counts()
# FN = 0

print(f"Accuracy = {TP/(TP+FP+FN):.4f}")
print(f"Precision = {TP/(TP+FP):.4f}")
print(f"Recall = {TP/(TP+FN):.4f}")

Accuracy = 0.7843
Precision = 0.8163
Recall = 0.9524


### North Sign (4)

In [ ]:
merged_df.loc[merged_df['labels']==3]['result'].value_counts()

result
TP    22
Name: count, dtype: int64

In [ ]:
TP = merged_df.loc[merged_df['labels']==3]['result'].value_counts()
FP, FN = 0, 0

print(f"Accuracy = {TP[0]/(TP[0]+FP+FN):.4f}")
print(f"Precision = {TP[0]/(TP[0]+FP):.4f}")
print(f"Recall = {TP[0]/(TP[0]+FN):.4f}")

Accuracy = 1.0000
Precision = 1.0000
Recall = 1.0000


C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15412\902526925.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Accuracy = {TP[0]/(TP[0]+FP+FN):.4f}")
C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15412\902526925.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Precision = {TP[0]/(TP[0]+FP):.4f}")
C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15412\902526925.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a valu

## Missing Labels in the Image

In [ ]:
# predictions_df.groupby('image')['labels'].unique()
labels_df = predictions_df.groupby('image')['labels'].apply(lambda x: set(x)).reset_index()

all_labels = {0, 1, 2, 3}
labels_df['missing_labels'] = labels_df['labels'].apply(lambda x: all_labels - x)

labels_df

,image,labels,missing_labels
0,11913d58-drawing_88.png,"{0, 1, 2, 3}",{}
1,17bd5f92-drawing_106.png,"{0, 1, 2, 3}",{}
2,2067578c-drawing_24.png,"{0, 2, 3}",{1}
3,2466f98b-drawing_67.png,"{0, 1, 2, 3}",{}
4,3108be30-drawing_43.png,"{0, 1, 2, 3}",{}
5,449d473c-drawing_25.png,"{0, 1, 2, 3}",{}
6,4fdca17b-drawing_7.png,"{0, 1, 2, 3}",{}
7,5acab2e6-drawing_64.png,"{0, 2, 3}",{1}
8,8318502d-drawing_15.png,"{0, 1, 2, 3}",{}
9,92e4c80c-drawing_45.png,"{0, 1, 2, 3}",{}


## Data Information

In [ ]:
# Ground Truth
print(f'Number of Labels in Training Data - {ground_truth_df.shape[0]}')
print(f'Bar Scale Count - {ground_truth_df.loc[ground_truth_df["labels"]==0].shape[0]}')
print(f'Color Stamp Count - {ground_truth_df.loc[ground_truth_df["labels"]==1].shape[0]}')
print(f'Detail Label Count - {ground_truth_df.loc[ground_truth_df["labels"]==2].shape[0]}')
print(f'North Sign Count - {ground_truth_df.loc[ground_truth_df["labels"]==3].shape[0]}')

Number of Labels in Training Data - 116
Bar Scale Count - 27
Color Stamp Count - 17
Detail Label Count - 50
North Sign Count - 22


In [ ]:
# Predictions
print(f'Number of Labels in Predictions - {predictions_df.shape[0]}')
print(f'Bar Scale Count - {predictions_df.loc[predictions_df["labels"]==0].shape[0]}')
print(f'Color Stamp Count - {predictions_df.loc[predictions_df["labels"]==1].shape[0]}')
print(f'Detail Label Count - {predictions_df.loc[predictions_df["labels"]==2].shape[0]}')
print(f'North Sign Count - {predictions_df.loc[predictions_df["labels"]==3].shape[0]}')

Number of Labels in Predictions - 116
Bar Scale Count - 28
Color Stamp Count - 17
Detail Label Count - 49
North Sign Count - 22


## Ground Truth & Predictions 
### Shape Comparison 

In [ ]:
ground_truth_df.shape

(116, 7)

In [ ]:
predictions_df.shape

(116, 7)

In [ ]:
merged_df.shape

(118, 13)

## Page-wise TP, FP, FN

In [ ]:
# pd.merge(ground_truth_df, predictions_df, on=['image'], how='outer', suffixes=('_gt', '_pred'))
merged_df

,x1_gt,y1_gt,x2_gt,y2_gt,labels,image,row_count,x1_pred,y1_pred,x2_pred,y2_pred,iou,result
0,626.232558,409.581395,732.186047,429.348837,0,11913d58-drawing_88.png,1,626.598572,407.394806,735.464539,434.808777,0.697665,TP
1,279.116279,529.767442,363.720930,559.023256,1,11913d58-drawing_88.png,1,277.574066,530.063721,364.678162,561.884888,0.876035,TP
2,626.232558,266.465116,713.209302,290.186047,2,11913d58-drawing_88.png,1,626.650024,266.320312,716.786926,295.447113,0.779139,TP
3,217.441860,518.697674,278.325581,542.418605,2,11913d58-drawing_88.png,2,215.104141,517.599670,284.647095,546.666199,0.714474,TP
4,46.651163,66.418605,83.813953,105.162791,3,11913d58-drawing_88.png,1,46.887444,67.970680,84.399574,105.122253,0.938615,TP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,223.767442,170.790698,275.953488,185.813953,2,f601361a-drawing_42.png,1,223.108826,169.652451,276.554596,189.965347,0.722160,TP
114,596.976744,180.279070,642.046512,195.302326,2,f601361a-drawing_42.png,2,595.494690,178.782028,643.956848,197.251953,0.756452,TP
115,242.744186,242.744186,294.139535,258.558140,2,f601361a-drawing_42.png,3,246.576401,244.471344,296.188232,259.973083,0.734807,TP
116,232.465116,408.000000,268.046512,423.813953,2,f601361a-drawing_42.png,4,228.863617,407.815247,272.935669,428.410400,0.619919,TP


In [ ]:
image_analysis_df = pd.DataFrame()
image_analysis_df['image'] = merged_df['image'].unique()
image_analysis_df['TP'] = merged_df.groupby('image')['result'].apply(lambda x: x.value_counts().get('TP', 0)).values
image_analysis_df['FP'] = merged_df.groupby('image')['result'].apply(lambda x: x.value_counts().get('FP', 0)).values
image_analysis_df['FN'] = merged_df.groupby('image')['result'].apply(lambda x: x.value_counts().get('FN', 0)).values

In [33]:
image_analysis_df.sort_values(by=['FP', 'FN'], ascending=False)

NameError: name 'image_analysis_df' is not defined

## Final Testing

In [34]:
image_data_actual = ground_truth['images/11913d58-drawing_88.png']
image_data_predicted = results['images\\11913d58-drawing_88.png']

In [35]:
for i in range(len(image_data_actual['labels'])):
    for j in range(len(image_data_predicted['labels'])):
        if calculate_iou(image_data_actual['boxes'][i], image_data_predicted['boxes'][j]) > 0.6:
            if image_data_actual['labels'][i] == image_data_predicted['labels'][j]:
                print(f'Actual Label - {image_data_actual["labels"][i]}, index - {i}')
                print(f'Predicted Label - {image_data_predicted["labels"][j]}, index - {j}')
                print("True Positive")
            elif image_data_actual['labels'][i] != image_data_predicted['labels'][j]:
                print(f'Actual Label - {image_data_actual["labels"][i]}, index - {i}')
                print(f'Predicted Label - {image_data_predicted["labels"][j]}, index - {j}')
                print("False Positive")

Actual Label - 3, index - 0
Predicted Label - 3, index - 1
True Positive
Actual Label - 3, index - 1
Predicted Label - 3, index - 4
True Positive
Actual Label - 1, index - 2
Predicted Label - 1, index - 0
True Positive
Actual Label - 1, index - 2
Predicted Label - 1, index - 2
True Positive
Actual Label - 2, index - 5
Predicted Label - 2, index - 3
True Positive


In [36]:
def final_result(image, ground_truth, results):
    TP, FP, FN = 0, 0, 0

    result_dict = []
    for i in range(len(ground_truth[f'images/{image}']['labels'])):
        label = ground_truth[f'images/{image}']['labels'][i]
        # label = 2 
        match = False
        for j in range(len(results[f'images\\{image}']['labels'])):
            pred_label = results[f'images\\{image}']['labels'][j].numpy()
            iou = calculate_iou(ground_truth[f'images/{image}']['boxes'][i], results[f'images\\{image}']['boxes'][j].numpy())
            if iou > 0.6:
                if pred_label == label:
                    result_dict.append({
                        'image': image,
                        'label': label, 
                        'ground_truth': ground_truth[f'images/{image}']['boxes'][i],
                        'prediction': results[f'images\\{image}']['boxes'][j].numpy(),
                        'iou': iou, 
                        'result': 'TP',                                      
                    })
                    # TP = TP + 1
                    match = True
                    continue
                elif pred_label != label:
                    result_dict.append({
                        'image': image,
                        'label': pred_label, 
                        'ground_truth': ground_truth[f'images/{image}']['boxes'][i],
                        'prediction': results[f'images\\{image}']['boxes'][j].numpy(),
                        'iou': iou, 
                        'result': 'FP',                
                    })
                    continue
            
        if match == False:
            result_dict.append({
                'image': image,
                'label': label, 
                'ground_truth': ground_truth[f'images/{image}']['boxes'][i],
                'prediction': [],
                'iou': 0, 
                'result': 'FN',                
            })
    return result_dict
                    

In [37]:
result_dict = final_result('eef413ab-drawing_22.png', ground_truth, results)
# pd.DataFrame(result_dict)

In [38]:
results

{'images\\11913d58-drawing_88.png': {'scores': tensor([0.6592, 0.7146, 0.7473, 0.8986, 0.9024]),
  'labels': tensor([1, 3, 1, 2, 3]),
  'boxes': tensor([[279.9822, 533.0856, 364.6894, 560.5040],
          [ 36.1040, 352.7266,  78.7248, 387.7281],
          [281.5031, 531.3296, 359.0376, 558.7917],
          [627.0461, 263.4313, 713.5956, 293.7730],
          [ 46.1654,  68.8650,  86.7926, 104.5161]])},
 'images\\17bd5f92-drawing_106.png': {'scores': tensor([0.6268, 0.8774, 0.8912, 0.7898, 0.9091, 0.8337, 0.7293, 0.6111]),
  'labels': tensor([1, 2, 1, 0, 2, 2, 1, 3]),
  'boxes': tensor([[610.9005, 386.0514, 697.9389, 423.2951],
          [425.3572, 105.8692, 499.8235, 138.2541],
          [613.1027, 384.3115, 706.1846, 423.0567],
          [ 43.2773, 510.1511, 138.2857, 537.2689],
          [427.0192, 275.1875, 534.4046, 305.8016],
          [ 73.6709, 258.9998, 171.9169, 287.2045],
          [617.1539, 384.5109, 705.5579, 419.8030],
          [592.7847,  92.5162, 632.7473, 127.0305]])}

In [39]:
new_df = pd.DataFrame()
for image in os.listdir('Temp/images'):
    result_dict = final_result(image, ground_truth, results)
    temp_df = pd.DataFrame(result_dict)
    
    new_df = pd.concat([new_df, temp_df], ignore_index=True)
    

In [40]:
new_df

,image,label,ground_truth,prediction,iou,result
0,11913d58-drawing_88.png,3,"[39.53488372093023, 351.06976744186045, 75.116...","[36.104023, 352.7266, 78.72478, 387.72806]",0.766419,TP
1,11913d58-drawing_88.png,3,"[46.65116279069767, 66.41860465116278, 83.8139...","[46.165363, 68.865, 86.79257, 104.51612]",0.847471,TP
2,11913d58-drawing_88.png,1,"[279.1162790697674, 529.7674418604652, 363.720...","[279.98224, 533.0856, 364.68942, 560.504]",0.827194,TP
3,11913d58-drawing_88.png,1,"[279.1162790697674, 529.7674418604652, 363.720...","[281.50308, 531.3296, 359.0376, 558.7917]",0.860245,TP
4,11913d58-drawing_88.png,0,"[626.2325581395348, 409.58139534883725, 732.18...",[],0.000000,FN
...,...,...,...,...,...,...
129,f601361a-drawing_42.png,0,"[57.72093023255814, 523.4418604651163, 162.883...","[57.11677, 521.8612, 164.94464, 547.4728]",0.903289,TP
130,f601361a-drawing_42.png,2,"[232.46511627906978, 407.99999999999994, 268.0...","[230.10313, 407.32843, 272.38864, 425.8176]",0.719705,TP
131,f601361a-drawing_42.png,2,"[242.74418604651163, 242.7441860465117, 294.13...","[243.70311, 245.83932, 290.78616, 261.50238]",0.629439,TP
132,f601361a-drawing_42.png,2,"[223.7674418604651, 170.7906976744186, 275.953...","[223.88919, 171.26787, 274.29764, 190.03215]",0.735720,TP


In [41]:
new_df['result'].value_counts()

result
TP    110
FN     21
FP      3
Name: count, dtype: int64

In [42]:
new_df['result'].value_counts()

result
TP    110
FN     21
FP      3
Name: count, dtype: int64

In [52]:
new_df['result'].value_counts()

result
TP    110
FN     21
FP      3
Name: count, dtype: int64

In [53]:
TP, FN, FP= new_df['result'].value_counts()
# FP = 0

In [54]:
print(f"Accuracy = {TP/(TP+FP+FN):.4f}")
print(f"Precision = {TP/(TP+FP):.4f}")
print(f"Recall = {TP/(TP+FN):.4f}")

Accuracy = 0.8209
Precision = 0.9735
Recall = 0.8397


### Classwise Accuracy

In [55]:
new_df.loc[new_df['label'] == 0]['result'].value_counts()

result
TP    18
FN     9
Name: count, dtype: int64

In [57]:
# Barscale 
TP, FN = new_df.loc[new_df['label'] == 0]['result'].value_counts()
FP = 0
# FN = 0
accuracy0 = TP/(TP+FP+FN)
print(f"Accuracy = {accuracy0:.4f}")
print(f"Precision = {TP/(TP+FP):.4f}")
print(f"Recall = {TP/(TP+FN):.4f}")


Accuracy = 0.6667
Precision = 1.0000
Recall = 0.6667


In [58]:
new_df.loc[new_df['label'] == 1]['result'].value_counts()

result
TP    31
Name: count, dtype: int64

In [60]:
# Color Stamp
TP = new_df.loc[new_df['label'] == 1]['result'].value_counts()
FN = 0
FP = 0
accuracy1= TP[0]/(TP[0]+FP+FN)
print(f"Accuracy = {accuracy1:.4f}")
print(f"Precision = {TP[0]/(TP[0]+FP):.4f}")
print(f"Recall = {TP[0]/(TP[0]+FN):.4f}")

Accuracy = 1.0000
Precision = 1.0000
Recall = 1.0000


C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15036\336505981.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  accuracy1= TP[0]/(TP[0]+FP+FN)
C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15036\336505981.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Precision = {TP[0]/(TP[0]+FP):.4f}")
C:\Users\Chirantan_Degloorkar\AppData\Local\Temp\ipykernel_15036\336505981.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, u

In [61]:
# Detail Labels
TP, FN, FP = new_df.loc[new_df['label'] == 2]['result'].value_counts()
accuracy2 = TP/(TP+FP+FN)
print(f"Accuracy = {accuracy2:.4f}")
print(f"Precision = {TP/(TP+FP):.4f}")
print(f"Recall = {TP/(TP+FN):.4f}")

Accuracy = 0.7736
Precision = 0.9318
Recall = 0.8200


In [62]:
# North Sign
TP, FN = new_df.loc[new_df['label'] == 3]['result'].value_counts()
FP = 0
accuracy3 = TP/(TP+FP+FN)
print(f"Accuracy = {accuracy3:.4f}")
print(f"Precision = {TP/(TP+FP):.4f}")
print(f"Recall = {TP/(TP+FN):.4f}")

Accuracy = 0.8696
Precision = 1.0000
Recall = 0.8696


In [63]:
# Model Accuracy 
print(f"Model Accuracy = {(accuracy0 + accuracy1 + accuracy2 + accuracy3)/4:.4f}")

Model Accuracy = 0.8275


In [64]:
image_analysis_df = pd.DataFrame()
image_analysis_df['image'] = new_df['image'].unique()
image_analysis_df['TP'] = new_df.groupby('image')['result'].apply(lambda x: x.value_counts().get('TP', 0)).values
image_analysis_df['FP'] = new_df.groupby('image')['result'].apply(lambda x: x.value_counts().get('FP', 0)).values
image_analysis_df['FN'] = new_df.groupby('image')['result'].apply(lambda x: x.value_counts().get('FN', 0)).values

In [65]:
## TP, FP, FN count by image
image_analysis_df

,image,TP,FP,FN
0,11913d58-drawing_88.png,5,0,2
1,17bd5f92-drawing_106.png,8,0,0
2,2067578c-drawing_24.png,3,1,2
3,2466f98b-drawing_67.png,10,0,0
4,3108be30-drawing_43.png,4,0,1
5,449d473c-drawing_25.png,9,0,1
6,4fdca17b-drawing_7.png,2,0,2
7,5acab2e6-drawing_64.png,5,0,0
8,8318502d-drawing_15.png,4,0,1
9,92e4c80c-drawing_45.png,8,0,0
